### Based off of RRL_parallelized.ipynb
##### Implements parallelization and TF graph execution with @tf.function

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
tf.config.list_physical_devices('GPU')

In [ ]:
# Define da model

from keras import layers

INPUT_SIZE = 30
LEARNING_RATE = 0.0001

input = layers.Input(shape=(INPUT_SIZE,))
x = layers.Dense(20, activation='relu')(input)
x = layers.Dense(10, activation='relu')(input)
x = layers.Dense(1, activation='tanh')(x)
F_curr_model = keras.Model(inputs=input, outputs=x)
F_prev_model = keras.Model(inputs=input, outputs=x)
F_curr_model.summary()

F_curr_model.compile(optimizer=keras.optimizers.Adam(LEARNING_RATE), loss='mse')

In [ ]:
# Following 2 functions take whole arrays, of shape (instances, size)

def gen_price_series(size=10000, k=3, a=0.9, instances=1):
    p_series = np.ndarray((instances, size,), dtype=np.float32)
    b_series = np.ndarray((instances, size,), dtype=np.float32)
    p_series[:, 0] = 0
    b_series[:, 0] = 0

    for i in range(1, size):
        p_series[:, i] =  p_series[:, i-1] + b_series[:, i-1] + k * np.random.normal(size=instances)
        b_series[:, i] = a * b_series[:, i-1] + np.random.normal(size=instances)

    # shape: (instances)
    R = np.max(p_series, axis=1) - np.min(p_series, axis=1)
    z_series = np.exp(p_series / np.repeat(R[:, np.newaxis], size, axis=1))

    return z_series

def calc_price_returns(zt):
    # returns the set rt, with the first element (0) being NaN
    rt = np.ndarray(zt.shape, dtype=np.float32)
    rt[:, 0] = np.nan
    rt[:, 1:] = zt[:, 1:] - zt[:, :-1]
    return rt



# Following 2 functions take slices of the arrays, of shape (instances,)

def calc_return(mu, rt, Ft_curr, Ft_prev, rft=tf.constant(0.0), delta=tf.constant(0.0)):
    '''Calculates the returns (Rt) on-line.'''

    return mu * (rft + Ft_prev * (rt - rft) - delta * tf.math.abs(Ft_curr - Ft_prev))

def calc_DSR(n, Rt, At_prev, Bt_prev):
    '''Calculates the differential Sharpe ratios (DSR, Dt) on-line.'''

    At_curr = At_prev + n * (Rt - At_prev)
    Bt_curr = Bt_prev + n * (tf.math.square(Rt) - Bt_prev)

    dDt_dRt = (Bt_prev - At_prev * Rt) / tf.math.pow((Bt_prev - tf.math.square(At_prev)), 1.5)

    return dDt_dRt, At_curr, Bt_curr


from math import prod
# Repeat grades of shape (instances,) to shape (instances,) + grad.shape
def reshape_grad(grad, shape):
    # takes a tf.Tensor grad of shape (instances,) and the target shape (variables)
    instances = grad.shape[0]
    total_elements = prod(shape)
    grad = tf.repeat(grad, int(total_elements / instances))
    grad = tf.reshape(grad, shape)
    
    return grad

In [ ]:
# Simulate trading using the outputs of the model
# Takes SLICES of the arrays, of shape (instances,)

def test_performance(zt, Ft):
    # zt: (instances, size)
    # Ft: (instances, size)

    instances = zt.shape[0]
    size = zt.shape[1]
    Ft = np.sign(Ft)

    values = np.ones((instances, size))
    owned = np.zeros((instances,))
    money = np.ones((instances,))
    values[:, 0] = money

    values_ideal = np.ones((instances, size))
    owned_ideal = np.zeros((instances,))
    money_ideal = np.ones((instances,))
    values_ideal[:, 0] = money_ideal

    for t in range(INPUT_SIZE, size - 1):

        # Model Ft

        # buy if Ft 1, owned 0 --> owned 1
        # sell if Ft -1, owned 1 --> owned 0

        # hold if Ft 0, owned 0 or 1
        # hold if Ft 1, owned 1
        # hold if Ft -1, owned 0

        # model
        buy = np.clip(Ft[:, t] * (1 - owned), 0, 1) # 1 if BUY, 0 if not
        sell = np.clip(-Ft[:, t] * owned, 0, 1) # 1 if SELL, 0 if not
        decision = buy - sell # 1 if BUY, -1 if SELL, 0 if HOLD
        owned = np.clip(owned + decision, 0, 1)
        money -= decision * zt[:, t]
        values[:, t] = money + owned * zt[:, t]

        # ideal
        deltas_ideal = np.sign(zt[:, t + 1] - zt[:, t])
        buy_ideal = np.clip(deltas_ideal * (1 - owned_ideal), 0, 1)
        sell_ideal = np.clip(-deltas_ideal * owned_ideal, 0, 1)
        decision_ideal = buy_ideal - sell_ideal
        owned_ideal = np.clip(owned_ideal + decision_ideal, 0, 1)
        money_ideal -= decision_ideal * zt[:, t]
        values_ideal[:, t] = money_ideal + owned_ideal * zt[:, t]

    return (values[-1] / zt[-1], values_ideal[-1] / zt[-1]), (values, values_ideal)


In [ ]:

@tf.function
def calc_grads(zt_curr, zt_prev, rt, At, Bt, N, MU, F_curr_model, F_prev_model):
    '''
    zt_curr: zt[:, t - INPUT_SIZE + 1:t + 1]
    zt_prev: zt[:, t - INPUT_SIZE:t]
    rt: rt[:, t]
    At: At[:, t-1]
    Bt: Bt[:, t-1]
    '''

    INSTANCES = zt_curr.shape[0]

    with tf.GradientTape(persistent=True) as tape:
        F_curr = tf.reshape(F_curr_model(zt_curr), (INSTANCES,))
        F_prev = tf.reshape(F_prev_model(zt_prev), (INSTANCES,)) # On the first iteration, this does not exist and is not used.
        

        # if the first iteration, F(t-1) does not yet exist so no update can be made.
        # F_prev_model.set_weights(F_curr_model.get_weights())
        # calculate the gradient
        Rt = calc_return(MU, rt, F_curr, F_prev)
        dDt_dRt, At_new, Bt_new = calc_DSR(N, Rt, At, Bt)

    # calculate derivatives.
    dRt_dFcurr = tape.gradient(Rt, F_curr) # shape: (instances,)
    dRt_dFprev = tape.gradient(Rt, F_prev) # shape: (instances,)
    dFcurr_dThetacurr = tape.jacobian(F_curr, F_curr_model.trainable_variables) # shape: (instances, MODEL VAR SHAPE)
    dFprev_dThetaprev = tape.jacobian(F_prev, F_prev_model.trainable_variables) # shape: (instances, MODEL VAR SHAPE)

    return F_curr, At_new, Bt_new, dDt_dRt, dRt_dFcurr, dRt_dFprev, dFcurr_dThetacurr, dFprev_dThetaprev

In [ ]:
# GENERATION VARS
import time

MU = tf.constant(3.0)
N = tf.constant(0.01)
RISKFREE_RETURN = 0
TRANS_COST = 0.1
SERIES_LENGTH = 1000
TRADING_DELAY = 100
K = 3
A = 0.9

EPISODES = 100
INSTANCES = 1

BASELINE_SERIES = gen_price_series(size=SERIES_LENGTH, k=K, a=A, instances=INSTANCES)

for ep in range(EPISODES):

    # generate price series
    zt = gen_price_series(size=SERIES_LENGTH, k=K, a=A, instances=INSTANCES)
    rt = calc_price_returns(zt)
    Ft = np.zeros((INSTANCES, SERIES_LENGTH,))

    # breaks if init at one; MUST INIT ZERO
    At = np.zeros((INSTANCES, SERIES_LENGTH,), dtype=np.float32)
    Bt = np.zeros((INSTANCES, SERIES_LENGTH,), dtype=np.float32)

    SR_series = np.zeros((INSTANCES, SERIES_LENGTH,))
    DSR_series = np.zeros((INSTANCES, SERIES_LENGTH,))
    Rt_series = np.zeros((INSTANCES, SERIES_LENGTH,))
    dD_series = np.ones((INSTANCES, SERIES_LENGTH,))
    autodiff_series = np.ones((INSTANCES, SERIES_LENGTH,))


    # plt.plot(zt)
    # plt.show()


    for t in range(INPUT_SIZE, SERIES_LENGTH): 
        Ft[:, t], At[:, t], Bt[:, t], dDt_dRt, dRt_dFcurr, dRt_dFprev, dFcurr_dThetacurr, dFprev_dThetaprev = calc_grads(zt[:, t - INPUT_SIZE + 1:t + 1], zt[:, t - INPUT_SIZE:t], rt[:, t], At[:, t-1], Bt[:, t-1], N, MU, F_curr_model, F_prev_model)

        # dont update parameters if DSR hasn't stabilized yet
        if t < TRADING_DELAY:
            continue

        # Set F(t-1) to F(t) for the next iteration.
        F_prev_model.set_weights(F_curr_model.get_weights())

        # print(len(F_curr_model.trainable_variables))
        
        if t != INPUT_SIZE:
            # multiply derivatives together.
            gradient_update = []

            for i in range(len(dFcurr_dThetacurr)):
                total_elements = prod(dFcurr_dThetacurr[i].shape)

                dDt_dRt_exp = reshape_grad(dDt_dRt, dFcurr_dThetacurr[i].shape)
                dRt_dFcurr_exp = reshape_grad(dRt_dFcurr, dFcurr_dThetacurr[i].shape)
                dRt_dFprev_exp = reshape_grad(dRt_dFprev, dFcurr_dThetacurr[i].shape)

                grad = dDt_dRt_exp * (dRt_dFcurr_exp * dFcurr_dThetacurr[i] + dRt_dFprev_exp * dFprev_dThetaprev[i])
                grad = tf.reduce_sum(grad, axis=0)
                grad *= LEARNING_RATE / INSTANCES # divide by instances since the gradients are summed over all instances.

                # gradient_update.append(tf.reshape(grad, F_curr_model.trainable_variables[i].shape))
                gradient_update.append(grad)

            vars = F_curr_model.trainable_variables
            for i in range(len(vars)):
                vars[i].assign_add(gradient_update[i])

    if ep % 1 == 0:
        print("Episode: ", ep)
        # test performance
        deltas, val_series = test_performance(zt, Ft)

        fig, ax = plt.subplots(2, figsize=(5, 3))

        ax[0].plot(zt[0])
        ax[0].plot(val_series[0][0])
        ax[0].set_title("price")

        ax[1].plot(Ft[0] * 0)
        ax[1].plot(Ft[0])
        ax[1].set_title("decision")
        plt.show()

